### 汇总和计算描述统计
Pandas对象拥有一组常用的数学和统计方法，它们大部分都属于约简和汇总统计。跟对应的Numpy数组方法相比，它们都是基于没有缺失数据的假设而构建的。
#### 约简方法
用于从Series中提取单个值（如sum或min）或从DataFrame的行或列中提取一个Series

In [2]:
import pandas as pn
import numpy as np

In [3]:
df = pn.DataFrame([[1.4,np.nan],[7.1,-4.5],[np.nan,np.nan],[0.75,-1.3]],index=['a','b','c','d'], columns=['one','two'])

In [4]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


调用**sum**方法，将返回一个含有列小计的Series

In [5]:
df.sum()

one    9.25
two   -5.80
dtype: float64

传入axis=1,将会按行进行求和

In [6]:
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

NA值将被自动排除，除非整行或列都是NA，可以通过skipna选项禁用该功能

In [9]:
df.max(axis=1, skipna = False)

a     NaN
b    7.10
c     NaN
d    0.75
dtype: float64

下表列出这些约简方法常用选项
![](images/20190629111510.jpg)

间接统计（比如获取最小值或最大值的索引的方法）

In [10]:
df.idxmax()

one    b
two    d
dtype: object

#### 累计方法

In [11]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


#### 非约简型也非累计型
汇总统计（describe方法）:一次性产生多个汇总统计：

In [12]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


对于非数值型数据：

In [14]:
obj = pn.Series(['a','b','c','a']*4)

In [15]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

下面列出所有与描述统计相关的方法
![](images/717869111fa967f658ac314.jpg)

#### 增长率（pct_change）
序列(Series)、DataFrame和Panel都有pct_change方法来计算增长率，作用是相比上一个数值的百分比变化量
#### DataFrame.iteritems()
该方法返回列名和序列的迭代器
#### DataFrame.head(n)
该方法返回最前n行数据
#### DataFrame.tail(n)
该方法返回最后n行
#### DataFrame.iterrows()
该方法返回索引和序列的迭代器
#### 相关系数和协方差
 有些汇总统计比如相关系数和协方差是由多个参数计算出的，这里使用pandas-datareader库获取包含股价和交易量的DataFrame，使用conda或pip进行安装
 
 pip install pandas-datareader或者conda install pandas-datareader

In [16]:
import pandas_datareader.data as web

In [22]:
all_data = {ticker: web.get_data_yahoo(ticker)
            for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}

In [26]:
price = pn.DataFrame({ticker:data['Adj Close']
                     for ticker, data in all_data.items()})

In [29]:
value = pn.DataFrame({ticker:data['Volume']
                     for ticker, data in all_data.items()})

计算股价的百分比变化（增长率）

In [30]:
returns = price.pct_change()

In [31]:
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2019-06-24,-0.001006,0.001078,0.005914,-0.005669
2019-06-25,-0.015158,-0.007104,-0.031572,-0.026149
2019-06-26,0.021629,0.001156,0.003747,-0.006029
2019-06-27,-0.000300,0.000000,0.001643,-0.003510
2019-06-28,-0.009112,-0.004476,-0.001416,0.004554


Series的**corr**方法用于计算两个Series中重叠的、非NA的、按索引对齐的值的相关系数，与此类似，**cov**用于计算协方差：

In [32]:
returns.MSFT.corr(returns.IBM)

0.48706627362064536

In [33]:
returns.MSFT.cov(returns.IBM)

8.670380875281656e-05

DataFrame的corr和cov将以DataFrame的形式返回完整的相关系数或协方差矩阵

In [34]:
returns.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.379346,0.453043,0.460606
IBM,0.379346,1.000000,0.487066,0.403020
MSFT,0.453043,0.487066,1.000000,0.537925
GOOG,0.460606,0.403020,0.537925,1.000000


In [35]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.000266,0.000076,0.000107,0.000116
IBM,0.000076,0.000151,0.000087,0.000077
MSFT,0.000107,0.000087,0.000209,0.000120
GOOG,0.000116,0.000077,0.000120,0.000239


利用DataFrame的corrwith方法，计算其列或行跟另一个Series或DataFrame之间的相关系数传入一个Series将返回一个相关系数值Series(针对各列进行计算)

In [36]:
returns.corrwith(returns.IBM)

AAPL    0.379346
IBM     1.000000
MSFT    0.487066
GOOG    0.403020
dtype: float64

传入一个DataFrame则会计算按列名配对的相关系数。计算百分比变化与成交量的相关系数

In [38]:
returns.corrwith(value)

AAPL   -0.062147
IBM    -0.156145
MSFT   -0.090059
GOOG   -0.020350
dtype: float64

传入axis=1即可按行进行计算

无论如何，在计算相关系数之前，所有的数据项都会按标签对齐
#### unique方法
unique方法可以得到Series中的唯一值数组

In [40]:
obj = pn.Series(['c','a','d','a','c','d','b','d','b'])

In [41]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

返回的唯一值是未排序的
#### values_counts方法
values_counts用于计算一个Series中各值出现的频率

In [42]:
obj.value_counts()

d    3
c    2
a    2
b    2
dtype: int64

结果Series是按值频率降序排列的，同时，value_counts还是一个顶级pandas方法，可用于任何数组或序列：

In [47]:
pn.value_counts(obj.values, sort=False)

b    2
a    2
d    3
c    2
dtype: int64

#### isin方法

isin（）用于判断矢量化集合的成员资格，可用于选取Series中或DataFrame列中数据的子集

In [48]:
mask = obj.isin(['b','d'])

In [49]:
mask

0    False
1    False
2     True
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [50]:
obj[mask]

2    d
5    d
6    b
7    d
8    b
dtype: object

下表给出这几个方法的一些参考信息
![](images/20190705145725.jpg)
#### DataFrame中的apply和applymap方法
要想对DataFrame中某些行或列进行操作，需要使用**apply()**方法，而想要对所有元素进行操作，就要使用**applymap()**方法

In [53]:
data = pn.DataFrame({'Qu1':[1,3,4,3,4], 'Qu2':[2,3,1,2,3], 'Qu3':[1,3,5,7,6]})

In [54]:
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,3
2,4,1,5
3,3,2,7
4,4,3,6


将value_counts方法传给apply方法：

In [57]:
results = data.apply(pn.value_counts).fillna(0)

In [58]:
results

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,0.0
3,2.0,2.0,1.0
4,2.0,0.0,0.0
5,0.0,0.0,1.0
6,0.0,0.0,1.0
7,0.0,0.0,1.0
